In [ ]:
from nltk import tokenize
from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
path ="/Users/pranoysarath/PycharmProjects/pythonProject1/AI_FInal_project/sequentiality_full_liwc.csv"
df = pd.read_csv(path)

In [ ]:
import spacy

# Load the spaCy English language model
nlp = spacy.load("en_core_web_sm")

def get_mood_proportion(sentence):
    # Process the sentence with spaCy
    doc = nlp(sentence)

    # Counters for realis and irrealis verbs
    realis_count = 0
    irrealis_count = 0

    # Loop through tokens in the sentence
    for token in doc:
        # Check if the token is a verb
        if token.pos_ == "VERB":
            # Check for past or present tense verbs
            if token.tag_ in ["VBD", "VBG", "VBN", "VBP", "VBZ"]:
                realis_count += 1
            else:
                irrealis_count += 1

    # Calculate the proportion of realis verbs
    total_verbs = realis_count + irrealis_count
    proportion_realis = realis_count / float(total_verbs) if total_verbs > 0 else 0

    return proportion_realis

In [ ]:
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
def get_bert_embedding(text):

    # Tokenize input text
    tokens = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)


    # Convert tokens to PyTorch tensor
    tokens_tensor = torch.tensor([tokens])

    # Forward pass, get hidden states
    with torch.no_grad():
        outputs = model(tokens_tensor)

    # Extract the hidden states (output embeddings) from the last layer
    last_hidden_states = outputs.last_hidden_state

    # For a single sentence input, take the embeddings for [CLS] token
    cls_embedding = last_hidden_states[:, 0, :]

    return cls_embedding.numpy()


In [ ]:
import spacy
import pandas as pd

# Load the pre-trained spaCy model
nlp = spacy.load("en_core_web_md")

# Load the Concreteness Ratings dataset
concreteness_df = pd.read_excel("/Users/pranoysarath/PycharmProjects/pythonProject1/AI_FInal_project/Concreteness_ratings_Brysbaert_et_al_BRM.xlsx")

def get_concreteness(word):
    # Check if the word is in the Concreteness Ratings dataset
    if word in concreteness_df['Word'].values:
        # Get the concreteness rating for the word
        concreteness = concreteness_df.loc[concreteness_df['Word'] == word, 'Conc.M'].values[0]
        return concreteness
    else:
        return None

def calculate_concrete_proportion(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Counters for concrete and abstract words
    concrete_count = 0
    total_words = 0

    # Iterate through tokens in the document
    for token in doc:
        # Consider only alphabetic tokens (ignore punctuation, numbers, etc.)
        if token.is_alpha:
            total_words += 1
            concreteness_score = get_concreteness(token.text.lower())

            # You can adjust the threshold based on your specific requirements
            # Here, I consider words with a concreteness score greater than 3 as concrete
            if concreteness_score is not None and concreteness_score > 3:
                concrete_count += 1

    # Calculate the proportion of concrete words
    concrete_proportion = concrete_count / total_words if total_words > 0 else 0
    return concrete_proportion

# Example usage
sample_text = "The cat sat on the mat and looked out of the window."
proportion = calculate_concrete_proportion(sample_text)

print(f"The proportion of concrete lexicon in the text is: {proportion}")


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
def sentiment(sentence):
    return sid.polarity_scores(sentence)


In [ ]:
def get_length(sentence):
    return len(sentence)

def get_count(sentence):
    return len(tokenize.word_tokenize(sentence))

def get_realis(sentence):
    return get_mood_proportion(sentence)
def get_bert(sentence):
    return get_bert_embedding(sentence)
def get_concrete(row):
    if int(row.name) % 100 == 0:
        print(f"concrete row count {row.name}")
    return calculate_concrete_proportion(row['story'])


df['sentence_length'] = df.apply(lambda row: get_length(row['story']), axis = 1)
df['word_count'] = df.apply(lambda row: get_count(row['story']), axis = 1)
df['realis_score'] = df.apply(lambda row: get_realis(row['story']), axis = 1)
df['concrete_score'] = df.apply(lambda row: get_concrete(row), axis = 1)

In [ ]:
def get_sentiment(row):
    if int(row.name) % 100 == 0:
        print(f"sentiment row count {row.name}")
    return sentiment(row['story'])['neu']

In [ ]:
import spacy

# Load the spaCy English language model
nlp = spacy.load("en_core_web_sm")

def get_mood_proportion(sentence):
    # Process the sentence with spaCy
    doc = nlp(sentence)

    # Counters for realis and irrealis verbs
    realis_count = 0
    irrealis_count = 0

    # Loop through tokens in the sentence
    for token in doc:
        # Check if the token is a verb
        if token.pos_ == "VERB":
            # Check for past or present tense verbs
            if token.tag_ in ["VBD", "VBG", "VBN", "VBP", "VBZ"]:
                realis_count += 1
            else:
                irrealis_count += 1

    # Calculate the proportion of realis verbs
    total_verbs = realis_count + irrealis_count
    proportion_realis = realis_count / float(total_verbs) if total_verbs > 0 else 0

    return proportion_realis